In [1]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from math import sqrt
from numpy import concatenate
from pandas import read_csv
from pandas import DataFrame
from pandas import concat
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM

%matplotlib inline
plt.style.use('seaborn')
from fbprophet import Prophet

Using TensorFlow backend.


In [14]:
dataset = pd.read_csv('./Chicago_Crimes_2012_to_2017_new.csv', error_bad_lines=False,index_col=0)

In [15]:
print('Dataset Shape before drop_duplicate : ', dataset.shape)

('Dataset Shape before drop_duplicate : ', (1048575, 13))


In [16]:
# dataset = dataset.head(100)
dataset.head(5)

,Primary_Type,Description,Location_Description,Arrest,Domestic,Beat,District,Ward,Community_Area,X_Coordinate,Y_Coordinate,Latitude,Longitude
Date,,,,,,,,,,,,,
5/3/16 23:40,BATTERY,DOMESTIC BATTERY SIMPLE,APARTMENT,True,True,1022,10.0,24.0,29.0,1154907.0,1893681.0,41.864073,-87.706819
5/3/16 21:40,BATTERY,DOMESTIC BATTERY SIMPLE,RESIDENCE,False,True,313,3.0,20.0,42.0,1183066.0,1864330.0,41.782922,-87.604363
5/3/16 23:31,PUBLIC PEACE VIOLATION,RECKLESS CONDUCT,STREET,False,False,1524,15.0,37.0,25.0,1140789.0,1904819.0,41.894908,-87.758372
5/3/16 22:10,BATTERY,SIMPLE,SIDEWALK,False,False,1532,15.0,28.0,25.0,1143223.0,1901475.0,41.885687,-87.749516
5/3/16 22:00,THEFT,$500 AND UNDER,RESIDENCE,False,True,1523,15.0,28.0,25.0,1139890.0,1901675.0,41.886297,-87.761751


# transform string data to number

In [18]:
# Primary_Type string to number
unique_dict = dict((c, i) for i, c in enumerate(dataset['Primary_Type'].unique()))
dataset['Primary_Type'] = [unique_dict[t] for d, t in dataset.Primary_Type.iteritems()]

# Description string to number
unique_dictx = dict((c, i) for i, c in enumerate(dataset['Description'].unique()))
dataset['Description'] = [unique_dictx[t] for d, t in dataset.Description.iteritems()]

# Location_Description string to number
unique_dictx = dict((c, i) for i, c in enumerate(dataset['Location_Description'].unique()))
dataset['Location_Description'] = [unique_dictx[t] for d, t in dataset.Location_Description.iteritems()]

# Arrest, Domestic bool True/False to 0/1
dataset.Arrest = dataset.Arrest.astype(int)
dataset.Domestic = dataset.Domestic.astype(int)

dataset.head()
dataset.to_csv('cc12-17_numeric.csv')

In [6]:
dataset.info()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 5/3/16 23:40 to 5/3/16 14:25
Data columns (total 13 columns):
Primary_Type            100 non-null int64
Description             100 non-null int64
Location_Description    100 non-null int64
Arrest                  100 non-null int64
Domestic                100 non-null int64
Beat                    100 non-null int64
District                100 non-null float64
Ward                    100 non-null float64
Community_Area          100 non-null float64
X_Coordinate            100 non-null float64
Y_Coordinate            100 non-null float64
Latitude                100 non-null float64
Longitude               100 non-null float64
dtypes: float64(7), int64(6)
memory usage: 10.9+ KB


In [7]:
# convert series to supervised learning

def series_to_supervised(data, n_in=1, n_out=1, dropnan=True):
    n_vars = 1 if type(data) is list else data.shape[1]
    df = DataFrame(data)
    cols, names = list(), list()
    # input sequence (t-n, ... t-1)
    for i in range(n_in, 0, -1):
        cols.append(df.shift(i))
        names += [('var%d(t-%d)' % (j + 1, i)) for j in range(n_vars)]
    # forecast sequence (t, t+1, ... t+n)
    for i in range(0, n_out):
        cols.append(df.shift(-i))  # print(df.shift(-i).head())
        if i == 0:
            names += [('var%d(t)' % (j + 1)) for j in range(n_vars)]
        else:
            names += [('var%d(t+%d)' % (j + 1, i)) for j in range(n_vars)]
    # put it all together
    agg = concat(cols, axis=1)
    agg.columns = names
    # drop rows with NaN values
    if dropnan:
        agg.dropna(inplace=True)
    return agg

In [8]:
values = dataset.values 

print values.shape

# integer encode direction
encoder = LabelEncoder()

# ensure all data is float
values = values.astype('float32')

# normalize features
scaler = MinMaxScaler(feature_range=(0, 1))
scaled = scaler.fit_transform(values)

# print(scaled[:5])

# frame as supervised learning
reframed = series_to_supervised(scaled, 1, 1)

# drop columns we don't want to predict
print(reframed.head(5))
reframed.drop(reframed.columns[[13,14,15,16,17,18,19,20,21,22,23,24]], axis=1, inplace=True)
print(reframed.head(5))

(100, 13)
   var1(t-1)  var2(t-1)  var3(t-1)  var4(t-1)  var5(t-1)  var6(t-1)  \
1   0.000000   0.000000   0.000000        1.0        1.0   0.375825   
2   0.000000   0.000000   0.045455        0.0        1.0   0.083333   
3   0.083333   0.033333   0.090909        0.0        0.0   0.582921   
4   0.000000   0.066667   0.136364        0.0        0.0   0.586221   
5   0.166667   0.100000   0.045455        0.0        1.0   0.582508   

   var7(t-1)  var8(t-1)  var9(t-1)  var10(t-1)    ...     var4(t)  var5(t)  \
1   0.375000   0.469388   0.368421    0.360952    ...         0.0      1.0   
2   0.083333   0.387755   0.539474    0.767370    ...         0.0      0.0   
3   0.583333   0.734694   0.315789    0.157188    ...         0.0      0.0   
4   0.583333   0.551020   0.315789    0.192320    ...         0.0      1.0   
5   0.583333   0.551020   0.315789    0.144215    ...         0.0      0.0   

    var6(t)   var7(t)   var8(t)   var9(t)  var10(t)  var11(t)  var12(t)  \
1  0.083333  0.0833

In [9]:
train=0.6
test=0.4
# split into train and test sets
values = reframed.values
n_train_hours = int(reframed.shape[0]*train)
train = values[:n_train_hours, :]
test = values[n_train_hours:, :]
# split into input and outputs
train_X, train_y = train[:, :-1], train[:, -1:]
test_X, test_y = test[:, :-1], test[:, -1:]
# reshape input to be 3D [samples, timesteps, features]
train_X = train_X.reshape((train_X.shape[0], 1, train_X.shape[1]))
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[1]))
print(train_X.shape, train_y.shape, test_X.shape, test_y.shape)
#print(train_y[:5])

((59, 1, 13), (59, 1), (40, 1, 13), (40, 1))


In [10]:
# design network
model = Sequential()
model.add(LSTM(75, input_shape=(train_X.shape[1], train_X.shape[2])))
model.add(Dense(100))
model.add(Dense(1))
# using mean_absolute_error loss function
model.compile(loss='mae', optimizer='rmsprop')

model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 75)                26700     
_________________________________________________________________
dense_1 (Dense)              (None, 100)               7600      
_________________________________________________________________
dense_2 (Dense)              (None, 1)                 101       
Total params: 34,401
Trainable params: 34,401
Non-trainable params: 0
_________________________________________________________________


In [11]:
# fit network
history = model.fit(train_X, train_y, epochs=70, batch_size=32, validation_data=(test_X, test_y), verbose=1, shuffle=False)

Train on 59 samples, validate on 40 samples
Epoch 1/70
59/59 [==============================] - 1s - loss: 0.4530 - val_loss: 0.2404
Epoch 2/70
59/59 [==============================] - 0s - loss: 0.2614 - val_loss: 0.2023
Epoch 3/70
59/59 [==============================] - 0s - loss: 0.1839 - val_loss: 0.2110
Epoch 4/70
59/59 [==============================] - 0s - loss: 0.1782 - val_loss: 0.2081
Epoch 5/70
59/59 [==============================] - 0s - loss: 0.1782 - val_loss: 0.2043
Epoch 6/70
59/59 [==============================] - 0s - loss: 0.1776 - val_loss: 0.2125
Epoch 7/70
59/59 [==============================] - 0s - loss: 0.1750 - val_loss: 0.2031
Epoch 8/70
59/59 [==============================] - 0s - loss: 0.1756 - val_loss: 0.2047
Epoch 9/70
59/59 [==============================] - 0s - loss: 0.1746 - val_loss: 0.1984
Epoch 10/70
59/59 [==============================] - 0s - loss: 0.1752 - val_loss: 0.2029
Epoch 11/70
59/59 [==============================] - 0s - loss: 0

In [12]:
test_X = test_X.reshape((test_X.shape[0], 1, test_X.shape[2]))
print(test_X.shape)

(40, 1, 13)


In [13]:
# make a prediction
yhat = model.predict(test_X)
print(yhat[:5])
test_X = test_X.reshape((test_X.shape[0], test_X.shape[2]))
print(test_y[:,-1:])
# invert scaling for forecast
inv_yhat = concatenate((yhat, test_X[:, 1:]), axis=1)
inv_yhat = scaler.inverse_transform(inv_yhat)
inv_yhat = inv_yhat[:,0]
print(inv_yhat[:5])
# invert scaling for actual
test_y = test_y.reshape((len(test_y), 1))
inv_y = concatenate((test_y, test_X[:, 1:]), axis=1)
inv_y = scaler.inverse_transform(inv_y)
inv_y = inv_y[:,0]
print(inv_y[:5])
# calculate RMSE
rmse = sqrt(mean_squared_error(inv_y, inv_yhat))
print('Test RMSE: %.3f' % rmse)

[[ 0.66418147]
 [ 0.65400076]
 [ 0.58043003]
 [ 0.63376999]
 [ 0.60490775]]
[[ 0.37057495]
 [ 0.30105591]
 [ 0.67575073]
 [ 0.68212891]
 [ 0.60794067]
 [ 0.01641846]
 [ 0.70611572]
 [ 0.83123779]
 [ 0.3682251 ]
 [ 0.13537598]
 [ 0.68225098]
 [ 0.52337646]
 [ 0.67056274]
 [ 0.75372314]
 [ 0.44430542]
 [ 0.50219727]
 [ 0.58724976]
 [ 0.29693604]
 [ 0.19805908]
 [ 0.81399536]
 [ 0.00421143]
 [ 0.3644104 ]
 [ 0.87896729]
 [ 0.50265503]
 [ 0.67355347]
 [ 0.3291626 ]
 [ 0.37298584]
 [ 0.77258301]
 [ 0.34701538]
 [ 0.47686768]
 [ 0.30343628]
 [ 0.76174927]
 [ 0.12121582]
 [ 0.87127686]
 [ 0.26898193]
 [ 0.45199585]
 [ 0.16680908]
 [ 0.08215332]
 [ 0.57049561]
 [ 0.53128052]]
[ 7.97017765  7.84800911  6.96516037  7.60523987  7.25889301]
[ 4.44689941  3.6126709   8.10900879  8.18554688  7.29528809]
Test RMSE: 3.185
